In [7]:
import librosa
from librosa import feature
import numpy as np
import pandas as pd

In [47]:
def extract_features(file):

    # check if normal or abnormal
    if 'abnormal' in file:
        operation=1
    else:
        operation=0
    # loading the file, getting y and sr (sample rate)

    y, sr = librosa.load(file)

    # Getting S and phase

    S, phase = librosa.magphase(librosa.stft(y=y))

    # Features for the DataFrame
# chroma features have pitch which is probably not usefull
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))

    chroma_cqt = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr))

    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr))
# to do
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
    melspectrogram_min = np.min(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
    melspectrogram_max = np.max(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
    melspectrogram_sum = librosa.feature.melspectrogram(y=y, sr=sr, S=S).sum()
    melspectrogram_corr= np.mean(np.corrcoef(librosa.feature.melspectrogram(y=y, sr=sr, S=S)))
    melspectrogram_std= np.std(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
# to check This feature is one of the most important method to extract a feature of an audio signal and is 
#used majorly whenever working on audio signals. The mel frequency cepstral coefficients (MFCCs) of a signal 
#are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral 
#envelope.
    
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr))
#rms when plotted similarities with melspectorgram
#normal rms mean higher than abnormal?
    rms = np.mean(librosa.feature.rms(y=y, S=S))
#spectral centroid computes weighted mean of the frequencies in the sound
#plot is line on melspectogram
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, S=S))
#Bandwidth(blue zone) is the difference between the upper and lower frequencies in a continuous band of frequencies
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr, S=S))
#needs further investigation, but makes continous data discrete in bins
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, S=S))
#Spectral flatness (or tonality coefficient) is a measure to quantify how much noise-like a sound is, as opposed to
#being tone-like 1. A high spectral flatness (closer to 1.0) indicates the spectrum is similar to white noise. 
#It is often converted to decibel.
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y, S=S))
#The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at 
#least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and 
#he bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent 
#to a value close to 1 (or 0).
#rolloff with rolloff coefficient 0.01 seems to be the same for (ab)normal
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, S=S))
#Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram
    poly_features = np.mean(librosa.feature.poly_features(y=y, sr=sr, S=S))
#probably chroma
    tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr))
#normal has higher zero crossing rate?
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y))
#divide in frequency bands could give more information bout difference normal and abnormal (BPM)
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr,
                       #    hop_length=hop_length)[0]
    # putting them into a dataframe
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    return pd.DataFrame({ 'melspectrogram' : [melspectrogram],'melspectrogram_min':[melspectrogram_min],
                             'melspectrogram_max':[melspectrogram_max], 'melspectrogram_sum':[melspectrogram_sum],
                             'melspectrogram_corr':[melspectrogram_corr] ,'melspectrogram_std':[melspectrogram_std] ,
                             'mfcc' : [mfcc], 'rms' : [rms],
                            'spectral_centroid' : [spectral_centroid], 'spectral_bandwidth' : [spectral_bandwidth],
                            'spectral_contrast' : [spectral_contrast], 'spectral_flatness ' : [spectral_flatness],
                            'spectral_rolloff' : [spectral_rolloff], 
                            'zero_crossing_rate' : [zero_crossing_rate],"mean harm": np.mean(y_harmonic),
                            "mean perc": [np.mean(y_percussive)],"max harm":[np.max(y_harmonic)],"max perc": [np.max(y_percussive)],
                             "min harm":[np.min(y_harmonic)], "min perc":[np.min(y_percussive)], 'normal(0)/abnormal(1)':[operation]})

In [48]:
import os
import sys
import time
# start measuring excecution time
start_time = time.time()
#create empty list that will be filled with pathnames
#find paths normal wav files
list_normal_6 = []
directory_normal_6 = "/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/"
for filename in os.listdir(directory_normal_6):
    file = f"{directory_normal_6}{filename}"
    list_normal_6.append(file)
list_normal_6.sort()
#find paths abnormal wav files
list_abnormal_6 = []
directory_abnormal_6 = "/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/abnormal/"
for filename in os.listdir(directory_abnormal_6):
    file = f"{directory_abnormal_6}{filename}"
    list_abnormal_6.append(file)
list_normal_6.sort()
#add normal to df
for wav_file in list_normal_6[:100]:
    df = extract_features(wav_file)
    if wav_file == list_normal_6[0]:
        df.to_csv('Librosa_features.csv')
    else:
        df.to_csv('Librosa_features.csv', mode='a', header=False)
#add abnormal to df
for wav_file in list_abnormal_6[:100]:
    df = extract_features(wav_file)
    df.to_csv('Librosa_features.csv', mode='a', header=False) 
print("--- %s seconds ---" % (time.time() - start_time))

--- 224.3029763698578 seconds ---


In [40]:
list_normal_6

['/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000000.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000001.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000002.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000003.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000004.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000005.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000006.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000007.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000008.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00/normal/00000009.wav',
 '/home/regis/Desktop/Sound Project/files/valve/6_dB_valve/valve/id_00

In [41]:
valve=pd.read_csv('/home/regis/Desktop/Sound Project/Codit-project-main/Librosa_features.csv')

In [42]:
valve.head()

,Unnamed: 0,melspectrogram,melspectrogram_min,melspectrogram_max,melspectrogram_sum,melspectrogram_corr,melspectrogram_std,mfcc,rms,spectral_centroid,...,spectral_flatness,spectral_rolloff,zero_crossing_rate,mean harm,mean perc,max harm,max perc,min harm,min perc,normal(0)/abnormal(1)
0,0,0.006300,6.218734e-10,0.090277,347.580261,[[ 1. 0.44970309 -0.03385457 ... -0.0...,0.010437,-15.342528,0.002967,1399.372425,...,0.000852,2847.104130,[[0.02246094 0.03613281 0.05029297 0.04199219 ...,-3.781356e-07,4.012865e-07,0.006918,0.070441,-0.006861,-0.060939,0
1,0,0.007236,7.803309e-10,0.087623,399.178070,[[ 1. 0.36962274 0.0264104 ... -0.0...,0.009354,-14.877153,0.002920,1765.622412,...,0.001012,3777.878084,[[0.02636719 0.04638672 0.06054688 0.06201172 ...,-1.201143e-07,-3.884470e-07,0.006952,0.070928,-0.006763,-0.060128,0
2,0,0.006186,6.056689e-10,0.092516,341.284851,[[ 1. 0.43585331 0.08430464 ... -0.0...,0.008715,-16.809891,0.002680,1716.821280,...,0.000950,3394.926803,[[0.10693359 0.14599609 0.17480469 0.13769531 ...,-3.535212e-07,-4.008645e-07,0.007342,0.070832,-0.007282,-0.060499,0
3,0,0.006994,6.636679e-10,0.097133,385.858826,[[1. 0.39588772 0.07987256 ... 0.01366...,0.009895,-15.353941,0.002953,1466.548926,...,0.000681,3035.257362,[[0.02685547 0.03710938 0.04492188 0.04248047 ...,-1.655043e-09,-1.397088e-07,0.006073,0.073961,-0.005455,-0.060447,0
4,0,0.005965,5.532336e-10,0.119931,329.072296,[[ 1. 0.34363696 0.02192941 ... -0.0...,0.010416,-15.100811,0.002925,1267.393978,...,0.000508,2481.789092,[[0.02685547 0.03613281 0.04248047 0.04199219 ...,-7.166033e-07,1.267004e-07,0.006912,0.070822,-0.006740,-0.063913,0


In [43]:
valve.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [44]:
valve.columns

Index(['melspectrogram', 'melspectrogram_min', 'melspectrogram_max',
       'melspectrogram_sum', 'melspectrogram_corr', 'melspectrogram_std',
       'mfcc', 'rms', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_contrast', 'spectral_flatness ', 'spectral_rolloff',
       'zero_crossing_rate', 'mean harm', 'mean perc', 'max harm', 'max perc',
       'min harm', 'min perc', 'normal(0)/abnormal(1)'],
      dtype='object')

In [45]:
valve.head()

,melspectrogram,melspectrogram_min,melspectrogram_max,melspectrogram_sum,melspectrogram_corr,melspectrogram_std,mfcc,rms,spectral_centroid,spectral_bandwidth,...,spectral_flatness,spectral_rolloff,zero_crossing_rate,mean harm,mean perc,max harm,max perc,min harm,min perc,normal(0)/abnormal(1)
0,0.006300,6.218734e-10,0.090277,347.580261,[[ 1. 0.44970309 -0.03385457 ... -0.0...,0.010437,-15.342528,0.002967,1399.372425,1635.526256,...,0.000852,2847.104130,[[0.02246094 0.03613281 0.05029297 0.04199219 ...,-3.781356e-07,4.012865e-07,0.006918,0.070441,-0.006861,-0.060939,0
1,0.007236,7.803309e-10,0.087623,399.178070,[[ 1. 0.36962274 0.0264104 ... -0.0...,0.009354,-14.877153,0.002920,1765.622412,1767.165689,...,0.001012,3777.878084,[[0.02636719 0.04638672 0.06054688 0.06201172 ...,-1.201143e-07,-3.884470e-07,0.006952,0.070928,-0.006763,-0.060128,0
2,0.006186,6.056689e-10,0.092516,341.284851,[[ 1. 0.43585331 0.08430464 ... -0.0...,0.008715,-16.809891,0.002680,1716.821280,1784.159467,...,0.000950,3394.926803,[[0.10693359 0.14599609 0.17480469 0.13769531 ...,-3.535212e-07,-4.008645e-07,0.007342,0.070832,-0.007282,-0.060499,0
3,0.006994,6.636679e-10,0.097133,385.858826,[[1. 0.39588772 0.07987256 ... 0.01366...,0.009895,-15.353941,0.002953,1466.548926,1585.731245,...,0.000681,3035.257362,[[0.02685547 0.03710938 0.04492188 0.04248047 ...,-1.655043e-09,-1.397088e-07,0.006073,0.073961,-0.005455,-0.060447,0
4,0.005965,5.532336e-10,0.119931,329.072296,[[ 1. 0.34363696 0.02192941 ... -0.0...,0.010416,-15.100811,0.002925,1267.393978,1510.835471,...,0.000508,2481.789092,[[0.02685547 0.03613281 0.04248047 0.04199219 ...,-7.166033e-07,1.267004e-07,0.006912,0.070822,-0.006740,-0.063913,0


In [46]:
valve.dtypes

melspectrogram           float64
melspectrogram_min       float64
melspectrogram_max       float64
melspectrogram_sum       float64
melspectrogram_corr       object
melspectrogram_std       float64
mfcc                     float64
rms                      float64
spectral_centroid        float64
spectral_bandwidth       float64
spectral_contrast        float64
spectral_flatness        float64
spectral_rolloff         float64
zero_crossing_rate        object
mean harm                float64
mean perc                float64
max harm                 float64
max perc                 float64
min harm                 float64
min perc                 float64
normal(0)/abnormal(1)      int64
dtype: object

## to reuse in necessary

In [20]:
def extract_features(file):

    # check if normal or abnormal
    if 'abnormal' in file:
        operation=1
    else:
        operation=0
    # loading the file, getting y and sr (sample rate)

    y, sr = librosa.load(file)

    # Getting S and phase

    S, phase = librosa.magphase(librosa.stft(y=y))

    # Features for the DataFrame
# chroma features have pitch which is probably not usefull
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))

    chroma_cqt = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr))

    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr))
# to do
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
# to check This feature is one of the most important method to extract a feature of an audio signal and is 
#used majorly whenever working on audio signals. The mel frequency cepstral coefficients (MFCCs) of a signal 
#are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral 
#envelope.

    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr))
#rms when plotted similarities with melspectorgram
#normal rms mean higher than abnormal?
    rms = np.mean(librosa.feature.rms(y=y, S=S))
#spectral centroid computes weighted mean of the frequencies in the sound
#plot is line on melspectogram
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, S=S))
#Bandwidth(blue zone) is the difference between the upper and lower frequencies in a continuous band of frequencies
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr, S=S))
#needs further investigation, but makes continous data discrete in bins
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, S=S))
#Spectral flatness (or tonality coefficient) is a measure to quantify how much noise-like a sound is, as opposed to
#being tone-like 1. A high spectral flatness (closer to 1.0) indicates the spectrum is similar to white noise. 
#It is often converted to decibel.
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y, S=S))
#The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at 
#least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and 
#he bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent 
#to a value close to 1 (or 0).
#rolloff with rolloff coefficient 0.01 seems to be the same for (ab)normal
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, S=S))
#Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram
    poly_features = np.mean(librosa.feature.poly_features(y=y, sr=sr, S=S))
#probably chroma
    tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr))
#normal has higher zero crossing rate?
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y))
#divide in frequency bands could give more information bout difference normal and abnormal (BPM)
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr,
                       #    hop_length=hop_length)[0]
    # putting them into a dataframe
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    return pd.DataFrame({'chroma_stft': [chroma_stft], 'chroma_cqt': [chroma_cqt], 'chroma_cens' : [chroma_cens], 
                            'melspectrogram' : [melspectrogram], 'mfcc' : [mfcc], 'rms' : [rms],
                            'spectral_centroid' : [spectral_centroid], 'spectral_bandwidth' : [spectral_bandwidth],
                            'spectral_contrast' : [spectral_contrast], 'spectral_flatness ' : [spectral_flatness],
                            'spectral_rolloff' : [spectral_rolloff], 'poly_features' : [poly_features], 
                            'tonnetz' : [tonnetz], 'zero_crossing_rate' : [zero_crossing_rate],"mean harm": np.mean(y_harmonic),
                            "mean perc": [np.mean(y_percussive)],"max harm":[np.max(y_harmonic)],"max perc": [np.max(y_percussive)],
                             "min harm":[np.min(y_harmonic)], "min perc":[np.min(y_percussive)], 'normal(0)/abnormal(1)':[operation]})